In [46]:
import pandas as pd
import numpy as np
import pickle
import keras
from keras.models import Model, Sequential
from keras.applications import resnet50, vgg16
from keras.preprocessing import image
from sklearn.model_selection import train_test_split, KFold
from keras import backend as K
from keras.layers import *
from keras import layers
from keras.callbacks import EarlyStopping, Callback
from keras.regularizers import l1, l2
# from skimage.io import imread
import matplotlib.pyplot as plt
import sklearn
import lightgbm as lgb
import time
import tensorflow as tf
import json
import os
from sklearn import *

K.set_image_data_format('channels_last')

path = '../Data/'

def read_image(imag_path, image_id):
    img = image.load_img(imag_path + image_id)
    img = image.img_to_array(img)
#     img= vgg16.preprocess_input(image.img_to_array(img))
    return img

/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/learning_curve.py:22: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the functions are moved. This module will be removed in 0.20
  DeprecationWarning)


In [32]:
# attr_list = pd.read_csv(path + '/DatasetA_train_20180813/attribute_list.txt', index_col = 0, sep = '\t', header = None)
# attr_list = attr_list.apply(lambda s: s[1].replace(' ', '_'), axis = 1)
# attributes_per_class = pd.read_csv(path + '/DatasetA_train_20180813/attributes_per_class.txt', 
#                                    index_col = 0, sep = '\t', header = None)
# adj_graph
# attr = np.array(list(class_id_emb_attr['attr']))
# adj_graph = sklearn.metrics.pairwise.pairwise_distances(attr, metric = 'l2')
# np.argsort(adj_graph[0])
# adj_graph
# class_id_emb_attr['attr'].values
with open(path + '../zero-shot-gcn/data/imagenet_graph.pkl', 'rb') as handle:
    imagenet_graph = pickle.load(handle)
with open(path + '../zero-shot-gcn/data/list/invdict_wordntext.json', 'r') as fp:
    words = json.load(fp)
class_names = class_id_emb_attr['class_name'].values
class_name_to_id = dict([(c, i) for i, c in enumerate(class_names)])
class_wns = [[]] * class_names.shape[0]
class_neighbor_wns = [[]] * class_names.shape[0]
words_array = np.asarray(words)
for i, word in enumerate(words):
    for w in word.split(', '):
        if w in class_name_to_id:
#             print (class_name_to_id[w], i)
            if len(class_wns[class_name_to_id[w]]) == 0:
                class_wns[class_name_to_id[w]] = []
            class_wns[class_name_to_id[w]].append(i)
#             print (class_wns)
            if len(class_neighbor_wns[class_name_to_id[w]]) == 0:
                class_neighbor_wns[class_name_to_id[w]] = []
            class_neighbor_wns[class_name_to_id[w]].extend(imagenet_graph[i])
#             print (word, words_array[imagenet_graph[i]])
#             print (i, w)
#             if w not in class_dict:
#                 class_dict[w] = 0
#             class_dict[w] += 1
# print (class_wns)
wn_to_class = dict()
for i, wns in enumerate(class_wns):
    for wn in wns:
        if wn not in wn_to_class:
            wn_to_class[wn] = []
        wn_to_class[wn].append(i)
        
adj_graph = scipy.eye(class_names.shape[0])
# for i, neighbor_wns in enumerate(class_neighbor_wns):
#     for j, wns in enumerate(class_wns):
#         if i == j:
#             continue
#         for neighbor in neighbor_wns:
#             if neighbor in wns:
#                 adj_graph[i, j] = 1
#                 adj_graph[j, i] = 1
# #                 print(class_names[i], class_names[j])
#                 break

def find_adj(root, current_wn, remain_depth):
    if current_wn in wn_to_class:
        for adj_class in wn_to_class[current_wn]:
            adj_graph[root][adj_class] = 1
            adj_graph[adj_class][root] = 1
    if remain_depth == 0:
        return
    for wn in imagenet_graph[current_wn]:
        find_adj(root, wn, remain_depth - 1)
        
for i, wns in enumerate(class_wns):
    for wn in wns:
        find_adj(i, wn, 2)
adj_graph

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [33]:
adj_graph.sum(axis = 1)

array([ 2.,  1.,  2.,  1.,  2.,  7., 14.,  4.,  2.,  2.,  4.,  6.,  2.,
        1.,  1.,  1.,  1.,  1.,  2.,  7.,  1.,  1.,  1.,  1.,  1.,  5.,
        1.,  4.,  1.,  1.,  2.,  1.,  2.,  1.,  3.,  6.,  2.,  1.,  1.,
        6.,  5.,  6.,  1.,  1.,  1.,  1.,  1.,  7.,  1.,  9.,  1.,  1.,
        4.,  2.,  1.,  5.,  1.,  1.,  1.,  1.,  9.,  1.,  2.,  1.,  1.,
        2.,  1.,  1.,  5.,  5.,  2.,  1.,  1.,  2.,  2.,  3.,  2., 41.,
        2.,  3., 33., 37., 33., 35., 33., 33., 33., 35., 33., 36., 33.,
       33., 33., 22.,  1., 20., 33., 33., 33., 33.,  1.,  3.,  1.,  2.,
        2.,  2.,  2.,  2.,  1.,  1.,  1.,  2.,  2.,  1.,  1.,  2.,  2.,
        2.,  9.,  3.,  3.,  1.,  1.,  1.,  1.,  3., 33.,  3.,  2.,  2.,
        1.,  2.,  2., 34.,  2.,  2.,  1.,  2.,  2.,  1.,  1.,  1.,  3.,
        3.,  2.,  1.,  1.,  1.,  2.,  2.,  7., 33.,  2.,  3.,  3.,  1.,
       37.,  3.,  2.,  5., 13.,  2.,  1.,  1.,  2.,  1.,  1.,  2.,  1.,
        6.,  5.,  2.,  3.,  1.,  2.,  2., 21.,  5., 35.,  3.,  2

In [3]:
# del setA_train_data, setB_train_data
with open(path + 'setB_class_id_emb_attr.pickle', 'rb') as handle:
    class_id_emb_attr = pickle.load(handle)
with open(path + '/setA_train_data.pickle', 'rb') as handle:
    setA_train_data = pickle.load(handle)
with open(path + '/setB_train_data.pickle', 'rb') as handle:
    setB_train_data = pickle.load(handle)
with open(path + 'setB_test_data.pickle', 'rb') as handle:
    test_data = pickle.load(handle)
    
train_data = setA_train_data.append(setB_train_data)
del setA_train_data, setB_train_data
with open(path + '/model_sub/6_12_24_16_ini128_growth32_inistride1_augdata_05077/flat_train_re_2018_09_24_03_07_15.pickle', 'rb') as handle:
    flat_train_re = pickle.load(handle)
train_data['target'] = list(flat_train_re)
with open(path + '/model_sub/6_12_24_16_ini128_growth32_inistride1_augdata_05077/flat_test_re_2018_09_24_03_07_15.pickle', 'rb') as handle:
    flat_test_re = pickle.load(handle)
test_data['target'] = list(flat_test_re)
# train_data.drop(columns = ['img_id'], inplace = True)

# with open(path + '/model_sub/6_12_24_16_ini64_growth32_inistride1_augdata_05010/train_data_2018_09_23_10_47_03.pickle', 'rb') as handle:
#     train_data = pickle.load(handle)
# with open(path + '/model_sub/6_12_24_16_ini64_growth32_inistride1_augdata_05010/test_data_2018_09_23_10_47_03.pickle', 'rb') as handle:
#     test_data = pickle.load(handle)

In [6]:
test_data.head()

,img_id,img,target
0,0003ae092034aa69da9782b2a3b4a15a.jpg,"[[[254.0, 254.0, 254.0], [254.0, 254.0, 254.0]...","[0.4807719, 0.04748677, 0.5795642, 0.0, 0.0314..."
1,000a5f73edc44c5d734ecf4ec9a183b8.jpg,"[[[103.0, 105.0, 102.0], [122.0, 124.0, 121.0]...","[0.0, 0.12412622, 0.22720738, 0.22865361, 0.12..."
2,000d8bf47339a2fab8a058a286b0cd77.jpg,"[[[198.0, 228.0, 116.0], [153.0, 189.0, 83.0],...","[0.8957928, 0.17921163, 0.26937371, 0.37168986..."
3,0012a4bdd0caae038defb2f634978e42.jpg,"[[[163.0, 134.0, 102.0], [134.0, 105.0, 73.0],...","[0.0, 0.1223318, 0.1739594, 0.042672273, 0.296..."
4,0015fa45b8d32bd2ede711292eb50b67.jpg,"[[[70.0, 79.0, 74.0], [63.0, 68.0, 64.0], [75....","[0.059360333, 0.24851952, 0.04202298, 0.088105..."


In [11]:
fast_text =  pd.read_csv(path + '/DatasetB_20180919/FoundInFastText', 
                        index_col = 0, sep = ' ', header = None)
fast_text.index.name = 'class_name'
fast_text_df = pd.DataFrame(index = fast_text.index)
fast_text_df['emb'] = fast_text.apply(lambda s: np.array([float(x) for x in s])[:300], axis = 1)

train_data = train_data.merge(fast_text_df, how = 'left', on = 'class_name')

train_data['emb'] = train_data.apply(lambda s: np.hstack([s['emb_x'], s['emb_y']]), axis = 1)

class_id_emb_attr = class_id_emb_attr.merge(fast_text_df, how = 'left', on = 'class_name')
class_id_emb_attr['emb'] = class_id_emb_attr.apply(lambda s: np.hstack([s['emb_x'], s['emb_y']]), axis = 1)

In [28]:
category = train_data['class_id'].unique()
category_dict = dict((category[i], i) for i in range(category.shape[0]))

# vgg_model = vgg16.VGG16(include_top=True, weights = None, input_shape=(64, 64, 3), classes = train_data['class_id'].unique().shape[0])


# with open(path + 'train_vgg16_img_div255.pickle', 'rb') as handle:
#     train_y = pickle.load(handle)

# input = Input(shape = (2048, ))
# dense = Dense(2048, activation="relu")(input)
# dense = Dropout(0.5)(dense)
# dense = Dense(2048, activation="relu")(input)
# dense = Dense(1024, activation="relu")(input)
# dense = Dense(category.shape[0], activation="softmax")(input)

# vgg_model = Model(input, dense)

# vgg_model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['categorical_accuracy'])

# print (vgg_model.summary())

# train_image_feature_map = np.reshape(vgg_model.predict(train_data_part_img, verbose=  1), (train_data_part_length, -1))
# with open(path + 'train_vgg16_img_div255.pickle', 'wb+') as handle:
#     pickle.dump(train_y, handle)
# with open(path + 'train_vgg16_img_div255.pickle', 'rb') as handle:
#     train_image_feature_map = pickle.load(handle)
# train_data['target'] = list(train_image_feature_map)

def extract_array_from_series(s):
    return np.asarray(list(s))

class DenseNet:
    """
    """
    def __init__(self, cat_max, blocks, weight_decay, kernel_initializer, reduction, init_filters, growth_rate):
        self.cat_max = cat_max
        self.blocks = blocks
        self.weight_decay = weight_decay
        self.kernel_initializer = kernel_initializer
        self.model = self.small_densenet(
                classes = self.cat_max,
                blocks = self.blocks, 
                weight_decay = self.weight_decay, 
                kernel_initializer = self.kernel_initializer,
                init_filters = init_filters,
                reduction = reduction,
                growth_rate = growth_rate)

    def dense_block(self, x, blocks, name, weight_decay = 1e-4, kernel_initializer = 'he_normal', growth_rate = None):
        """A dense block.
        # Arguments
            x: input tensor.
            blocks: integer, the number of building blocks.
            name: string, block label.
        # Returns
            output tensor for the block.
        """
        for i in range(blocks):
            x = self.conv_block(x, growth_rate, name=name + '_block' + str(i + 1), 
                weight_decay = weight_decay,
                kernel_initializer = kernel_initializer)
        return x

    def transition_block(self, x, reduction, name, weight_decay = 1e-4, kernel_initializer = 'he_normal'):
        """A transition block.
        # Arguments
            x: input tensor.
            reduction: float, compression rate at transition layers.
            name: string, block label.
        # Returns
            output tensor for the block.
        """
        bn_axis = 3 if K.image_data_format() == 'channels_last' else 1
        x = layers.BatchNormalization(axis=bn_axis, epsilon=1.001e-5,
                                    name=name + '_bn')(x)
        x = layers.Activation('relu', name=name + '_relu')(x)
        x = layers.Conv2D(int(K.int_shape(x)[bn_axis] * reduction), 1,
                        use_bias=False,
                        kernel_initializer = kernel_initializer,
                        kernel_regularizer = l2(weight_decay),
                        name=name + '_conv')(x)
        x = layers.AveragePooling2D(2, strides=2, name=name + '_pool')(x)
        return x

    def conv_block(self, x, growth_rate, name, weight_decay = 1e-4, kernel_initializer = 'he_normal'):
        """A building block for a dense block.
        # Arguments
            x: input tensor.
            growth_rate: float, growth rate at dense layers.
            name: string, block label.
        # Returns
            Output tensor for the block.
        """
        bn_axis = 3 if K.image_data_format() == 'channels_last' else 1
        x1 = layers.BatchNormalization(axis=bn_axis,
                                    epsilon=1.001e-5,
                                    name=name + '_0_bn')(x)
        x1 = layers.Activation('relu', name=name + '_0_relu')(x1)
        x1 = layers.Conv2D(4 * growth_rate, 1,
                        use_bias=False,
                        kernel_initializer = kernel_initializer,
                        kernel_regularizer = l2(weight_decay),
                        name=name + '_1_conv')(x1)
        x1 = layers.BatchNormalization(axis=bn_axis, epsilon=1.001e-5,
                                    name=name + '_1_bn')(x1)
        x1 = layers.Activation('relu', name=name + '_1_relu')(x1)
        x1 = layers.Conv2D(growth_rate, 3,
                        padding='same',
                        use_bias=False,
                        kernel_initializer = kernel_initializer,
                        kernel_regularizer = l2(weight_decay),
                        name=name + '_2_conv')(x1)
        x = layers.Concatenate(axis=bn_axis, name=name + '_concat')([x, x1])
        return x

    def small_densenet(self, classes = None, 
                       img_input_shape = (64, 64, 3), 
                       blocks = [6, 12, 24, 16], 
                       weight_decay = 1e-4, 
                       kernel_initializer = 'he_normal',
                       reduction = 0.5, 
                       init_filters = None, 
                       growth_rate = None):
        img_input = Input(shape = (img_input_shape))
#         x = layers.ZeroPadding2D(padding=((3, 3), (3, 3)))(img_input)
#         x = layers.Conv2D(64, 3, strides=1, use_bias=False, 
#             kernel_initializer = kernel_initializer, 
#             kernel_regularizer = l2(weight_decay),
#             name='conv1/conv')(img_input)
#         x = layers.BatchNormalization(
#             axis=3, epsilon=1.001e-5, name='conv1/bn')(x)
#         x = layers.Activation('relu', name='conv1/relu')(x)
#         x = layers.ZeroPadding2D(padding=((1, 1), (1, 1)))(x)
#         x = layers.MaxPooling2D(3, strides=2, name='pool1')(x)
        
        x = layers.ZeroPadding2D(padding=((3, 3), (3, 3)))(img_input)
        x = layers.Conv2D(init_filters, 3, strides=1, use_bias=False, 
            kernel_initializer = kernel_initializer, 
            kernel_regularizer = l2(weight_decay),
            name='conv1/conv')(x)
        x = layers.BatchNormalization(
            axis=3, epsilon=1.001e-5, name='conv1/bn')(x)
        x = layers.Activation('relu', name='conv1/relu')(x)
        x = layers.ZeroPadding2D(padding=((1, 1), (1, 1)))(x)
        x = layers.AveragePooling2D(3, strides=2, name='pool1')(x)
        
        for i, block in enumerate(blocks):
            scope_num_str = str(i + 2)
            x = self.dense_block(x, block, name='conv' + scope_num_str, 
                                 weight_decay = weight_decay, kernel_initializer = kernel_initializer, growth_rate = growth_rate)
            if i != len(blocks) - 1:
                x = self.transition_block(x, reduction, name='pool' + scope_num_str, 
                                          weight_decay = weight_decay, kernel_initializer = kernel_initializer)
        
        x = layers.BatchNormalization(
            axis=3, epsilon=1.001e-5, name='bn')(x)
        x = layers.Activation('relu', name='relu')(x)

        x = layers.GlobalAveragePooling2D(name='avg_pool')(x)
        # x = Lambda(lambda x: x, name = 'densenet_features')(x)
        x = layers.Dense(classes, activation='softmax',
            kernel_initializer = kernel_initializer, 
#             kernel_regularizer = l2(weight_decay), 
            name='fc')(x)
        
        model = Model(img_input, x)
        print (model.summary())
        model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['categorical_accuracy'])
        
        return model
# train_data = train_data[:1000]
train_img = extract_array_from_series(train_data['img'])
train_img = vgg16.preprocess_input(train_img)
OneHotEncoder = sklearn.preprocessing.OneHotEncoder()
train_target = train_data['class_id'].apply(lambda id: category_dict[id]).values
train_target = OneHotEncoder.fit_transform(np.reshape(train_target, (-1, 1))).toarray()

In [ ]:
fold = 5
kf = KFold(n_splits=fold, shuffle=True)
num_fold = 0
batch_size = 256

for train_index, test_index in kf.split(train_img):
    train_part_img = train_img[train_index]
    validate_part_img = train_img[test_index]
    
    train_part_target = train_target[train_index]
    validate_part_target = train_target[test_index]
    
    print ('Train target nunique: ', np.unique(np.argwhere(train_part_target == 1)[:, 1]).shape[0], 
           'Validate target nuique: ', np.unique(np.argwhere(validate_part_target == 1)[:, 1]).shape[0])
#     model = vgg16.VGG16(include_top=True, weights = None, input_shape=(64, 64, 3), classes = category.shape[0])
#     model = resnet50.ResNet50(include_top=True, weights = None, input_shape=(224, 224, 3), classes = category.shape[0])
#     print (model.summary())
#     model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['categorical_accuracy'])

In [83]:
    img_model = DenseNet(blocks = [6, 12, 16, 16], 
                                cat_max = train_target.shape[1],
                                weight_decay = 1e-4, 
                                kernel_initializer = 'glorot_normal',
                                reduction = 0.8,
                                init_filters = 128,
                                growth_rate = 32) 
#     print (img_model.summary())
    # img_model = small_vgg(img_input_shape = (64, 64, 3), classes = train_target.shape[1])
#     datagen = keras.preprocessing.image.ImageDataGenerator(
# #             featurewise_center=True,
# #             featurewise_std_normalization=True,
#             rotation_range=45,
#             shear_range = 0.2,
#             zoom_range=0.2,
# #             width_shift_range=0.2,
# #             height_shift_range=0.2,
#             horizontal_flip=True)
#     datagen.fit(train_part_img)
#     h = img_model.model.fit_generator(datagen.flow(train_part_img, train_part_target, batch_size=batch_size), 
#                   validation_data=(validate_part_img, validate_part_target), 
#                   epochs=100, shuffle=True, verbose = 1, workers=1, use_multiprocessing=False, 
#                   callbacks=[EarlyStopping(monitor='val_categorical_accuracy', patience=25, verbose=0)],
#                   steps_per_epoch = train_part_img.shape[0]//batch_size,)
    h = img_model.model.fit(train_part_img, train_part_target, validation_data=(validate_part_img, validate_part_target), 
                  epochs=100, batch_size = 256, shuffle=True, verbose = 1, 
                  callbacks=[EarlyStopping(monitor='val_categorical_accuracy', patience=15, verbose=0)])
    break

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_33 (InputLayer)           (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
zero_padding2d_9 (ZeroPadding2D (None, 70, 70, 3)    0           input_33[0][0]                   
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 68, 68, 128)  3456        zero_padding2d_9[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 68, 68, 128)  512         conv1/conv[0][0]                 
__________________________________________________________________________________________________
conv1/relu

KeyboardInterrupt: 

In [20]:
tmp_model_dir = path + "./model_dir/6_12_24_16_ini_64_grow_32_03535/"
# time_label = time.strftime('_%Y_%m_%d_%H_%M_%S', time.gmtime())
# if not os.path.isdir(tmp_model_dir):
#     os.makedirs(tmp_model_dir, exist_ok=True)
# model_name = tmp_model_dir + "model" + time_label + ".h5"
# img_model.model.save(model_name)

In [21]:
pd.Series(category).to_csv(tmp_model_dir + 'category.csv')

In [5]:
def small_dnn(classes = None, 
                       img_input_shape = (64, 64, 3), 
                       blocks = [6, 12, 24, 16], 
                       weight_decay = 1e-4, 
                       kernel_initializer = 'he_normal',
                       reduction = 0.5):
    img_input = Input(shape = (img_input_shape))
    x = layers.Dense(classes, activation='softmax',
        kernel_initializer = kernel_initializer, 
#             kernel_regularizer = l2(weight_decay), 
        name='fc')(img_input)

    model = Model(img_input, x)
    print (model.summary())
    model.compile(optimizer = keras.optimizers.Adam(lr=1e-4), loss = 'categorical_crossentropy', metrics = ['categorical_accuracy'])

    return model

fold = 5
kf = KFold(n_splits=fold, shuffle=True, random_state = 100)
num_fold = 0
batch_size = 256
train_image_feature_map = extract_array_from_series(train_data['target'])

for train_index, test_index in kf.split(train_img):
    train_part_img = train_image_feature_map[train_index]
    validate_part_img = train_image_feature_map[test_index]
    
    train_part_target = train_target[train_index]
    validate_part_target = train_target[test_index]
    
    print ('Train target nunique: ', np.unique(np.argwhere(train_part_target == 1)[:, 1]).shape[0], 
           'Validate target nuique: ', np.unique(np.argwhere(validate_part_target == 1)[:, 1]).shape[0])

    img_classifi_model = small_dnn(img_input_shape = (1024, ), 
                                classes = train_target.shape[1]) 
    
    h = img_classifi_model.fit(train_part_img, train_part_target, validation_data=(validate_part_img, validate_part_target), 
                  epochs=100, batch_size = 64, shuffle=True, verbose = 1, 
                  callbacks=[EarlyStopping(monitor='val_categorical_accuracy', patience=300, verbose=0)])
    break

Train target nunique:  205 Validate target nuique:  205
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1024)              0         
_________________________________________________________________
fc (Dense)                   (None, 205)               210125    
Total params: 210,125
Trainable params: 210,125
Non-trainable params: 0
_________________________________________________________________
None
Train on 69799 samples, validate on 17450 samples
Epoch 1/100
69799/69799 [==============================] - 6s 80us/step - loss: 3.9231 - categorical_accuracy: 0.2365 - val_loss: 2.9243 - val_categorical_accuracy: 0.4150
Epoch 2/100
69799/69799 [==============================] - 5s 79us/step - loss: 2.4646 - categorical_accuracy: 0.4957 - val_loss: 2.1961 - val_categorical_accuracy: 0.5364
Epoch 3/100
69799/69799 [==============================] - 5s 78us/step - los

KeyboardInterrupt: 

In [12]:
pred_class_probas = img_classifi_model.predict(train_image_feature_map, verbose = 1)

38221/38221 [==============================] - 2s 45us/step


In [84]:
pred_class_max_proba = pred_class_probas.max(axis = 1)
pred_class_id = category[pred_class_probas.argmax(axis = 1)]
# np.sum(pred_class_id == train_data['class_id']) / train_data.shape[0]
train_data['pred_max_proba'] = pred_class_max_proba
train_data['pred_class_id'] = pred_class_id
# pred_class_max_proba

In [44]:
for th in np.linspace(0.5, 1, 10):
    precision = np.sum((train_data.pred_max_proba > th) & (train_data.pred_class_id == train_data.class_id)) \
        / np.sum(train_data.pred_max_proba > th)    
    print (th, precision)

0.5 0.973545127192473
0.5555555555555556 0.9791893883566691
0.6111111111111112 0.9839496930678245
0.6666666666666666 0.9879055597867479
0.7222222222222222 0.9910592325841301
0.7777777777777778 0.9933481858688733
0.8333333333333333 0.9950834818187818
0.8888888888888888 0.9968510066164243
0.9444444444444444 0.9981974428840914
1.0 nan


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in long_scalars
  


In [146]:
train_data['preds'] = list(pred_class_probas)

In [151]:
category_dict[('ZJL1', 'ZJL1']]

TypeError: unhashable type: 'list'

In [186]:
# np.asarray(list(train_data['img'])).shape
# train_data[train_data['class_id'] == 'ZJL1']
# train_data['class_id'].value_counts().hist()
# train_data.head()
# class_id_emb_attr.iloc[0].name
# resnet50_model = ResNet50(weights='imagenet', input_shape=(224, 224, 3))
img = image.load_img(path + '/DatasetA_train_20180813/train/000c0d617f5b67d116dee15c40d1d47d.jpeg', target_size=(224, 224))
img = image.img_to_array(img)
img = vgg16.preprocess_input(img)
preds = vgg_model.predict(np.expand_dims(img, axis=0))
print('Predicted:', vgg16.decode_predictions(preds, top=3)[0])
# imread(path + '/DatasetA_train_20180813/train/000c0d617f5b67d116dee15c40d1d47d.jpeg')
# image.img_to_array(img).shape

Predicted: [('n03598930', 'jigsaw_puzzle', 0.023861792), ('n02095314', 'wire-haired_fox_terrier', 0.022931756), ('n04589890', 'window_screen', 0.021361042)]


In [8]:
def lgbm_train(train_part, train_part_label, valide_part, valide_part_label, fold_seed,
        fold = 5, train_weight = None, valide_weight = None, flags = None):
    """
    LGBM Training
    """
    print("-----LGBM training-----")

    d_train = lgb.Dataset(train_part, train_part_label) #, categorical_feature = CATEGORY_FEATURES) #, init_score = train_part[:, -1])
    d_valide = lgb.Dataset(valide_part, valide_part_label) #, categorical_feature = CATEGORY_FEATURES) #, init_score = valide_part[:, -1])
    params = {
            'task': 'train',
            'boosting_type': 'gbdt', #'gbdt',
            'objective': 'regression',
            'metric': 'rmse',
            # 'num_leaves':-1,
            'min_sum_hessian_in_leaf':None,
            'max_depth':7,
            'learning_rate':0.005,
            'feature_fraction':0.1,
            'verbose':-1,
            'num_boost_round':3000,
            'drop_rate':None,
            'bagging_fraction':0.6,
            'bagging_freq':5,
            'early_stopping_round':100,
            # 'min_data_in_leaf':100,
            'max_bin': None,
            'scale_pos_weight':None,
        }
    print ("lightgbm params: {0}\n".format(params))

    bst = lgb.train(
                    params ,
                    d_train,
                    verbose_eval = 200,
                    valid_sets = [d_train, d_valide],
                    # feature_name= keras_train.DENSE_FEATURE_LIST,
                    #feval = gini_lgbm
                    #num_boost_round = 1
                    )
    #pred = model_eval(bst, 'l', valide_part)
    #print(pred[:10])
    #print(valide_part_label[:10])
    #print(valide_part[:10, -1])
    # exit(0)
    feature_imp = bst.feature_importance(importance_type = 'gain')
    sort_ind = np.argsort(feature_imp)[::-1]
    print (np.c_[np.array(FEATURE_LIST)[sort_ind], feature_imp[sort_ind]][:10])
    # print (np.array(keras_train.FEATURE_LIST)[np.argsort(feature_imp)])
    # exit(0)
    # cv_result = lgb.cv(params, d_train, nfold=fold) #, feval = gini_lgbm)
    # pd.DataFrame(cv_result).to_csv('cv_result', index = False)
    # exit(0)
    return bst

In [41]:
# train_data = setA_train_data

# img_model = DenseNet(blocks = [6, 12, 24, 16], 
#                                 cat_max = 205,
#                                 weight_decay = 1e-4, 
#                                 kernel_initializer = 'glorot_normal',
#                                 reduction = 0.5, 
#                                 init_filters = 64, 
#                                 growth_rate = 32).model
# img_model.load_weights(path + '/model_sub/6_12_24_16_ini64_growth32_inistride2_03621/model_0_2018_09_21_21_10_59.h5')
# img_model_flat = Model(input = img_model.input, output = img_model.get_layer(name = 'avg_pool').output)
# train_data['target'] = list(img_model_flat.predict(preprocess_img(train_data['img']), verbose = 1))
# train_data['preds'] = list(img_model.predict(train_img, verbose = 1))
# train_data['target'] = list(train_y) #
with open(path + '/model_sub/6_12_24_16_ini64_growth32_inistride2_03621/flat_train_re_2018_09_21_21_10_59.pickle', 'rb') as handle:
    flat_train_re = pickle.load(handle)
# train_data['target'] = list(flat_train_re)

In [43]:
flat_train_re[0]

array([1.0262023 , 1.0496653 , 0.42650044, ..., 0.09345146, 0.14242867,
       0.        ], dtype=float32)

In [40]:
train_data['target'].iloc[0]

array([2.2266190e+00, 1.6327184e-01, 2.8384233e-01, ..., 4.1941926e-04,
       3.3683911e-02, 7.6473856e-01], dtype=float32)

In [33]:


train_part_img_id = pd.read_csv(path + '/model_sub/6_12_24_16_ini64_growth16_02962/train_part_img_id_0.csv', header = None)
validate_part_img_id = pd.read_csv(path + '/model_sub/6_12_24_16_ini64_growth16_02962/validate_part_img_id_0.csv', header = None)
train_part_img_id = train_part_img_id[0].values
validate_part_img_id = validate_part_img_id[0].values

train_part_df = train_data[train_data['img_id'].isin(train_part_img_id)]
validate_part_df = train_data[train_data['img_id'].isin(validate_part_img_id)]

seen_class = train_part_df.append(validate_part_df).class_id.unique()

train_part_df = train_data[train_data['class_id'].isin(seen_class)]
validate_part_df = train_data[~train_data['class_id'].isin(seen_class)]
# unseen_class_df = train_data[~train_data['class_id'].isin(seen_class)]
unseen_class = validate_part_df.class_id.unique()

# validate_part_df = validate_part_df.append(unseen_class_df)
# train_part_df = train_part_df.append(validate_part_df)
# validate_part_df = unseen_class_df

train_part_data = create_dnn_data(train_part_df)
train_part_target = extract_array_from_series(train_part_df['target'])

validate_part_data = create_dnn_data(validate_part_df)
validate_part_target = extract_array_from_series(validate_part_df['target'])

In [22]:
setB_train_data[setB_train_data.class_id.isin(seen_class)].shape

(39184, 6)

In [33]:
4 / 10.77032961

0.3713906765013109

In [30]:
x = np.random.randint(1, 10, (4,3))
print (x)
print (np.linalg.norm(x, axis = 1))
sklearn.preprocessing.normalize(x, norm='l2')

[[8 6 4]
 [8 7 1]
 [2 2 5]
 [2 1 5]]
[10.77032961 10.67707825  5.74456265  5.47722558]


array([[0.74278135, 0.55708601, 0.37139068],
       [0.74926865, 0.65561007, 0.09365858],
       [0.34815531, 0.34815531, 0.87038828],
       [0.36514837, 0.18257419, 0.91287093]])

In [89]:
import scipy
        
def find_nearest_class(class_id_emb_attr, model, eval_df, cand_feature_map, img_feature_map, 
                       threshold = None, gamma = None, seen_class = None):
#     cand_feature_map = model.predict(create_dnn_data(class_id_emb_attr))
    # img_feature_map = np.repeat(img_feature_map, [cand_feature_map.shape[0]] * img_feature_map.shape[0], axis = 0)
    nearest_class_id = ['ZJL'] * eval_df.shape[0]
#     seen_class_bias = np.zeros(class_id_emb_attr.shape[0])
#     seen_class_bias[class_id_emb_attr.class_id.isin(seen_class)] = gamma
    
#     seen_indice = [category_dict[c] for c in seen_class]
#     preds = np.asarray(list(eval_df['preds']))[:, seen_indice]
    for i in range(img_feature_map.shape[0]):
        if False: #np.amax(preds[i]) > threshold:
            nearest_class_id[i] = seen_class[np.argmax(preds[i])]
        else:
            img = img_feature_map[i]
#             dis = 1 - sklearn.metrics.pairwise.cosine_similarity([img], 
#                                     cand_feature_map).reshape((cand_feature_map.shape[0]))
            dis = np.linalg.norm(img - cand_feature_map, axis = 1)
#             print (dis.shape)
#             if np.amin(dis[class_id_emb_attr.class_id.isin(seen_class)]) > \
#                     gamma * np.amin(dis[~class_id_emb_attr.class_id.isin(seen_class)]):
#                 dis[class_id_emb_attr.class_id.isin(seen_class)] = 200
#             dis += seen_class_bias
#             print(dis)
#             plt.hist(dis)
#             return
            min_ind = np.where(dis == np.amin(dis))[0]
            if len(min_ind) > 1:
                print ('eval img id: ', eval_df.iloc[i]['class_id'], 'has multiple best candidates: ', len(min_ind), 'min val: ', np.amin(dis))
    #         print (i, img, class_id_emb_attr.iloc[min_ind[0]].name)
            nearest_class_id[i] = class_id_emb_attr.iloc[min_ind[0]]['class_id']
#     print (nearest_class_id)
    return np.asarray(nearest_class_id)
        
def calc_accuracy(eval_df, eval_class, preds):
    eval_mask = eval_df.class_id.isin(eval_class)
    eval_num = np.sum(eval_mask)
    right_num = np.sum(preds[eval_mask] == eval_df.class_id[eval_mask])
    return right_num / np.sum(eval_mask), right_num, eval_num
    
def calc_detailed_accuracy(eval_df, preds, seen_class, unseen_class):
    all_re = calc_accuracy(eval_df, eval_df['class_id'].values, preds)
    seen_re = calc_accuracy(eval_df, seen_class, preds)
    unseen_re = calc_accuracy(eval_df, unseen_class, preds)
    print("\nAll_re: \t%.6f\t%.0f\t%.0f" % all_re)
    print("Seen_re: \t%.6f\t%.0f\t%.0f" % seen_re)
    print("Unseen_re: \t%.6f\t%.0f\t%.0f" % unseen_re)

def model_eval(model, model_type, eval_df, cand_class_id_emb_attr, img_feature_map, seen_class = None, unseen_class = None):
    if model_type == 'DEM':
        zs_model = Model(inputs = model.inputs[:2], outputs = model.outputs[0])
        cand_feature_map = zs_model.predict(create_dnn_data(cand_class_id_emb_attr))
    elif model_type == 'GCN':
        zs_model = Model(inputs = model.inputs[2:], outputs = model.outputs[0])
        cand_class_to_id = [class_to_id[c] for c in cand_class_id_emb_attr.class_id.values]
        cand_feature_map = zs_model.predict(None, steps = 1)[cand_class_to_id]
    preds = find_nearest_class(cand_class_id_emb_attr, zs_model, eval_df, cand_feature_map, 
                               img_feature_map)
    if 'class_id' in eval_df.columns:
        calc_detailed_accuracy(eval_df, preds, seen_class, unseen_class)
    return preds

def models_eval(models, eval_df, cand_class_id_emb_attr, img_feature_map, 
                seen_class = None, unseen_class = None):
    preds = []
    for model, model_type in models:
        pred = model_eval(model, model_type, eval_df, cand_class_id_emb_attr, img_feature_map, 
                          seen_class, unseen_class)
        preds.append(pred)
    return preds
        
def multi_models_vote(models, eval_df = None, cand_class_id_emb_attr = None, img_feature_map = None, 
                      seen_class = None, unseen_class = None):
    preds = models_eval(models, eval_df, cand_class_id_emb_attr, img_feature_map, 
                seen_class, unseen_class)
    preds = np.asarray(preds).T
    print (preds)
    vote_preds = []
    for single_img_vote in preds:
        uniq_val, counts = np.unique(single_img_vote, return_counts = True)
        vote_preds.append(uniq_val[np.argmax(counts)])
    vote_preds = np.asarray(vote_preds)
    print (vote_preds)
    if 'class_id' in eval_df.columns: 
        calc_detailed_accuracy(eval_df, vote_preds, seen_class, unseen_class)
    return vote_preds
    
class AccuracyEvaluation(Callback):
    def __init__(self, validation_data=(), interval=1, batch_interval = 1000000, verbose = 2, \
            scores = [], class_id_emb_attr = None, eval_df = None, threshold = None, \
                 seen_class = None, unseen_class = None, gamma = None, model_type = None):
        super(Callback, self).__init__()

        self.interval = interval
        self.X_val, self.y_val = validation_data
        # print("y_val shape:{0}".format(self.y_val.shape))
        self.batch_interval = batch_interval
        self.verbose = verbose
        self.scores = scores
        self.class_id_emb_attr = class_id_emb_attr
        self.eval_df = eval_df
        self.threshold = threshold
        self.seen_class = seen_class
        self.unseen_class = unseen_class
        self.gamma = gamma
        self.model_type = model_type
#         seen_indice = [category_dict[c] for c in seen_class]
#         self.preds = np.asarray(list(eval_df['preds']))[:, seen_indice]
#         self.img_feature_map = self.y_val
        
    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            model_eval(self.model, self.model_type, self.eval_df, self.class_id_emb_attr, 
                seen_class = self.seen_class, unseen_class = self.unseen_class, img_feature_map = self.y_val)
    
    def on_batch_end(self, batch, logs={}):
        return
        if(self.verbose >= 2) and (batch % self.batch_interval == 0):
            # y_pred = self.model.predict(self.X_val, verbose=0)
            # loss = metrics.log_loss(self.y_val, y_pred)
            print("Hi! on_batch_end() , batch=",batch,",logs:",logs)
            # print("Valide size=",y_pred.shape[0], "  Valide loss=",loss)

def full_connect_layer(input, hidden_dim, activation, resnet = False, adj_graphs = None, 
                       drop_out_ratio = None, kernel_initializer = 'he_normal'):
    full_connect = input
    for i, hn in enumerate(hidden_dim):
        fc_in = full_connect
        if drop_out_ratio is not None:
            full_connect = Dropout(drop_out_ratio)(full_connect)
        full_connect = BatchNormalization(epsilon=1.001e-5)(full_connect)
        # full_connect = Dense(hn, kernel_regularizer = l2(0.001), activity_regularizer = l1(0.001))(full_connect)
#         full_connect = Concatenate()([Dense(hn, kernel_initializer='lecun_uniform', activation = 'relu')(full_connect), 
#             Dense(hn, kernel_initializer='lecun_uniform', activation = 'sigmoid')(full_connect)])
        full_connect = Dense(hn, kernel_initializer=kernel_initializer, kernel_regularizer = l2(1e-4), activation = activation)(full_connect)
#         full_connect = LeakyReLU(alpha=0.02)(full_connect)
        # full_connect = self.act_blend(full_connect)
        # if self.full_connect_dropout > 0:
#             full_connect = Dropout(self.full_connect_dropout)(full_connect) #Dropout(self.full_connect_dropout)(full_connect)
        if adj_graphs is not None:
            full_connect = Lambda(lambda x: K.dot(x[1], x[0]), \
                                  name = 'rela_' + str(i))([full_connect, adj_graphs])
        if resnet:
            full_connect = Concatenate()([fc_in, full_connect])
    return full_connect

def create_dnn_data(df):
    # return [extract_array_from_series(df['attr']), extract_array_from_series(df['emb'])[:, :]]
    return [extract_array_from_series(df['attr']), extract_array_from_series(df['emb'])[:, :]]

In [84]:
print ('SVD...')
svd = decomposition.TruncatedSVD(n_components=10, n_iter=50, random_state=12)
svd_features = svd.fit_transform(extract_array_from_series(class_id_emb_attr['attr']))
class_id_emb_attr['svd_attr'] = list(svd_features)
train_data = train_data.merge(class_id_emb_attr[['class_id', 'svd_attr']], how = 'left', on = 'class_id')

SVD...


In [86]:
train_data.shape

(87249, 10)

In [94]:
def create_dnn(kernel_initializer = 'he_normal'):
    alpha = 0.03
    img_flat_len = 1032
    attr_input = Input(shape = (30,), name = 'attr')
    word_emb = Input(shape = (600,), name = 'wv')
    imag_classifier = Input(shape = (img_flat_len,), name = 'img')

    attr_dense = Dense(300, use_bias = False, kernel_initializer=kernel_initializer, 
                       kernel_regularizer = l2(1e-4))(attr_input)
#     attr_dense = Dense(512)(attr_dense)
#     attr_dense = LeakyReLU(alpha=alpha)(attr_dense)
#     attr_dense = Dense(256)(attr_dense)
#     attr_dense = LeakyReLU(alpha=alpha)(attr_dense)
#     attr_dense = Dense(300, activation="relu")(attr_dense)
    # attr_dense = Dense(512, activation="relu")(attr_dense)
    word_emb_dense = Dense(300, use_bias = False, kernel_initializer=kernel_initializer, 
                           kernel_regularizer = l2(1e-4))(word_emb)

    attr_word_emb = Concatenate()([word_emb, attr_input])
#     attr_word_emb = word_emb #Add()([word_emb_dense, attr_dense])
    attr_word_emb_dense = full_connect_layer(attr_word_emb, hidden_dim = [int(img_flat_len * 1.5), 
                                                                          int(img_flat_len * 1.25), 
                                                                          int(img_flat_len * 1.125),
#                                                                           int(img_flat_len * 0.5)
                                                                         ], \
                                             activation = 'relu', resnet = False, drop_out_ratio = 0.2)
    attr_word_emb_dense = full_connect_layer(attr_word_emb_dense, hidden_dim = [img_flat_len], 
                                             activation = 'relu')

    mse_loss = K.mean(keras.losses.mean_squared_error(imag_classifier, attr_word_emb_dense))
    
    model = Model([attr_input, word_emb, imag_classifier], outputs = attr_word_emb_dense) #, vgg_output])
    model.add_loss(mse_loss)
    model.compile(optimizer=keras.optimizers.Adam(lr=1e-4), loss=None)
#     model.summary()
    return model

kf = KFold(n_splits=5, shuffle=True, random_state = 100)
num_fold = 0
zs_model_list = []
classes = train_data.class_id.unique()

for train_index, test_index in kf.split(classes):
    seen_class = classes[train_index]
    unseen_class = classes[test_index]
    
    train_part_df = train_data[train_data.class_id.isin(seen_class)]
    validate_part_df = train_data[train_data.class_id.isin(unseen_class)]
    
    train_part_data = create_dnn_data(train_part_df)
    validate_part_data = create_dnn_data(validate_part_df)
    
    train_part_target = extract_array_from_series(train_part_df['target']) #sklearn.preprocessing.normalize(extract_array_from_series(train_part_df['target']), norm='l2')
    validate_part_target = extract_array_from_series(validate_part_df['target']) #sklearn.preprocessing.normalize(extract_array_from_series(validate_part_df['target']), norm='l2')

#     print ('Train Validation Classes: ', train_part_df.class_id.unique().shape[0], 
#            validate_part_df.class_id.unique().shape[0])
    print ('Seen unseen Classes: ', seen_class.shape[0], unseen_class.shape[0])

    callbacks = [
            EarlyStopping(monitor='val_loss', patience=1, verbose=0),
            AccuracyEvaluation(validation_data=(validate_part_data,  validate_part_target), interval=1, \
    #                         class_id_emb_attr = class_id_emb_attr, \
                               eval_df = validate_part_df, threshold= 0.3, \
                              seen_class = seen_class, unseen_class = unseen_class, \
                class_id_emb_attr = class_id_emb_attr[class_id_emb_attr.class_id.isin(unseen_class)], \
                              gamma = 0.8, model_type = 'DEM')
            ]
#     for i in range(5):
    zs_model = create_dnn(kernel_initializer = 'he_normal')
    if num_fold == 0:
        print (zs_model.summary())
    zs_model.fit(train_part_data + [train_part_target],  validation_data = (validate_part_data + [validate_part_target], None),
                  epochs=50, batch_size = 32, shuffle=True, verbose = 1, callbacks=callbacks)
    zs_model_list.append((zs_model, 'DEM'))
    num_fold += 1

Seen unseen Classes:  164 41


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:34: UserWarning: Output "dense_209" missing from loss dictionary. We assume this was done on purpose, and we will not be expecting any data to be passed to "dense_209" during training.


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
wv (InputLayer)                  (None, 600)           0                                            
____________________________________________________________________________________________________
attr (InputLayer)                (None, 30)            0                                            
____________________________________________________________________________________________________
concatenate_46 (Concatenate)     (None, 630)           0           wv[0][0]                         
                                                                   attr[0][0]                       
____________________________________________________________________________________________________
dropout_82 (Dropout)             (None, 630)           0           concatenate_46[0][0]    

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:41: RuntimeWarning: invalid value encountered in long_scalars


70304/70408 [============================>.] - ETA: 0s - loss: 0.5615
All_re: 	0.184787	3112	16841
Seen_re: 	nan	0	0
Unseen_re: 	0.184787	3112	16841
70408/70408 [==============================] - 31s - loss: 0.5613 - val_loss: 0.4646
Epoch 3/50
70400/70408 [============================>.] - ETA: 0s - loss: 0.3840
All_re: 	0.204798	3449	16841
Seen_re: 	nan	0	0
Unseen_re: 	0.204798	3449	16841
70408/70408 [==============================] - 31s - loss: 0.3840 - val_loss: 0.3882
Epoch 4/50
70368/70408 [============================>.] - ETA: 0s - loss: 0.3384
All_re: 	0.212458	3578	16841
Seen_re: 	nan	0	0
Unseen_re: 	0.212458	3578	16841
70408/70408 [==============================] - 31s - loss: 0.3384 - val_loss: 0.3639
Epoch 5/50
70400/70408 [============================>.] - ETA: 0s - loss: 0.3227
All_re: 	0.201057	3386	16841
Seen_re: 	nan	0	0
Unseen_re: 	0.201057	3386	16841
70408/70408 [==============================] - 31s - loss: 0.3227 - val_loss: 0.3561
Epoch 6/50
70304/70408 [=======

...

Train on 69141 samples, validate on 18108 samples
Epoch 1/50
69088/69141 [============================>.] - ETA: 0s - loss: 1.0362
All_re: 	0.137508	2490	18108
Seen_re: 	nan	0	0
Unseen_re: 	0.137508	2490	18108
69141/69141 [==============================] - 33s - loss: 1.0360 - val_loss: 0.7791
Epoch 2/50
69088/69141 [============================>.] - ETA: 0s - loss: 0.5712
All_re: 	0.189750	3436	18108
Seen_re: 	nan	0	0
Unseen_re: 	0.189750	3436	18108
69141/69141 [==============================] - 31s - loss: 0.5711 - val_loss: 0.4829
Epoch 3/50
69024/69141 [============================>.] - ETA: 0s - loss: 0.3853eval img id:  ZJL16 has multiple best candidates:  2 min val:  16.767563

All_re: 	0.187597	3397	18108
Seen_re: 	nan	0	0
Unseen_re: 	0.187597	3397	18108
69141/69141 [==============================] - 31s - loss: 0.3852 - val_loss: 0.4012
Epoch 4/50
69088/69141 [============================>.] - ETA: 0s - loss: 0.3368
All_re: 	0.188756	3418	18108
Seen_re: 	nan	0	0
Unseen_re: 	0.

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:34: UserWarning: Output "dense_227" missing from loss dictionary. We assume this was done on purpose, and we will not be expecting any data to be passed to "dense_227" during training.


Train on 69739 samples, validate on 17510 samples
Epoch 1/50
69728/69739 [============================>.] - ETA: 0s - loss: 1.0279
All_re: 	0.122159	2139	17510
Seen_re: 	nan	0	0
Unseen_re: 	0.122159	2139	17510
69739/69739 [==============================] - 33s - loss: 1.0279 - val_loss: 0.7599
Epoch 2/50
69728/69739 [============================>.] - ETA: 0s - loss: 0.5630
All_re: 	0.145631	2550	17510
Seen_re: 	nan	0	0
Unseen_re: 	0.145631	2550	17510
69739/69739 [==============================] - 31s - loss: 0.5630 - val_loss: 0.4722
Epoch 3/50
69696/69739 [============================>.] - ETA: 0s - loss: 0.3843
All_re: 	0.144774	2535	17510
Seen_re: 	nan	0	0
Unseen_re: 	0.144774	2535	17510
69739/69739 [==============================] - 31s - loss: 0.3843 - val_loss: 0.3953
Epoch 4/50
69600/69739 [============================>.] - ETA: 0s - loss: 0.3380
All_re: 	0.144032	2522	17510
Seen_re: 	nan	0	0
Unseen_re: 	0.144032	2522	17510
69739/69739 [==============================] - 31s - lo

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:34: UserWarning: Output "dense_233" missing from loss dictionary. We assume this was done on purpose, and we will not be expecting any data to be passed to "dense_233" during training.


Train on 68297 samples, validate on 18952 samples
Epoch 1/50
68224/68297 [============================>.] - ETA: 0s - loss: 1.0336
All_re: 	0.144048	2730	18952
Seen_re: 	nan	0	0
Unseen_re: 	0.144048	2730	18952
68297/68297 [==============================] - 33s - loss: 1.0333 - val_loss: 0.7701
Epoch 2/50
68224/68297 [============================>.] - ETA: 0s - loss: 0.5680
All_re: 	0.177132	3357	18952
Seen_re: 	nan	0	0
Unseen_re: 	0.177132	3357	18952
68297/68297 [==============================] - 31s - loss: 0.5678 - val_loss: 0.4793
Epoch 3/50
68160/68297 [============================>.] - ETA: 0s - loss: 0.3870
All_re: 	0.192645	3651	18952
Seen_re: 	nan	0	0
Unseen_re: 	0.192645	3651	18952
68297/68297 [==============================] - 31s - loss: 0.3869 - val_loss: 0.3985
Epoch 4/50
68192/68297 [============================>.] - ETA: 0s - loss: 0.3397
All_re: 	0.185838	3522	18952
Seen_re: 	nan	0	0
Unseen_re: 	0.185838	3522	18952
68297/68297 [==============================] - 31s - lo

In [103]:
for i, model in enumerate(zs_model_list):
    model_name = path + '/model_sub/zs_model_' + str(i) + time_label + ".txt"
    zs_model_list[i][0].save(model_name)

In [15]:
multi_models_vote(models = zs_model_list, eval_df = validate_part_df, \
            cand_class_id_emb_attr = class_id_emb_attr[class_id_emb_attr.class_id.isin(unseen_class)], \
            img_feature_map = validate_part_target, seen_class = seen_class, unseen_class = unseen_class)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:41: RuntimeWarning: invalid value encountered in long_scalars



All_re: 	0.213811	3570	16697
Seen_re: 	nan	0	0
Unseen_re: 	0.213811	3570	16697

All_re: 	0.227226	3794	16697
Seen_re: 	nan	0	0
Unseen_re: 	0.227226	3794	16697

All_re: 	0.235911	3939	16697
Seen_re: 	nan	0	0
Unseen_re: 	0.235911	3939	16697

All_re: 	0.216566	3616	16697
Seen_re: 	nan	0	0
Unseen_re: 	0.216566	3616	16697

All_re: 	0.070492	1177	16697
Seen_re: 	nan	0	0
Unseen_re: 	0.070492	1177	16697
[['ZJL200' 'ZJL200' 'ZJL200' 'ZJL19' 'ZJL101']
 ['ZJL4' 'ZJL100' 'ZJL41' 'ZJL4' 'ZJL101']
 ['ZJL192' 'ZJL192' 'ZJL192' 'ZJL192' 'ZJL101']
 ...
 ['ZJL261' 'ZJL102' 'ZJL254' 'ZJL254' 'ZJL261']
 ['ZJL254' 'ZJL254' 'ZJL261' 'ZJL254' 'ZJL137']
 ['ZJL261' 'ZJL261' 'ZJL261' 'ZJL254' 'ZJL261']]
['ZJL200' 'ZJL4' 'ZJL192' ... 'ZJL254' 'ZJL254' 'ZJL261']

All_re: 	0.234533	3916	16697
Seen_re: 	nan	0	0
Unseen_re: 	0.234533	3916	16697


array(['ZJL200', 'ZJL4', 'ZJL192', ..., 'ZJL254', 'ZJL254', 'ZJL261'],
      dtype='<U6')

In [54]:
x = np.random.randint(1, 10, (4, 3))
x

array([[8, 5, 9],
       [8, 7, 7],
       [1, 1, 7],
       [7, 1, 9]])

In [99]:
uniq_val, counts = np.unique(x, return_counts = True)
print (uniq_val, counts)
# uniq_val[np.argmax(counts)]
np.argmin(counts)

[1 5 7 8 9] [3 1 4 2 2]


1

In [39]:
attr = np.array(list(class_id_emb_attr['attr']))
# adj_graph = scipy.eye(attr.shape[0]) #1 - sklearn.metrics.pairwise.pairwise_distances(attr, metric = 'cosine')
adj_graph = 1 - sklearn.metrics.pairwise.pairwise_distances(
    np.array(list(class_id_emb_attr['emb'])), metric = 'cosine')
# th = 0.99999
# adj_graph[adj_graph > th] = 1
# adj_graph[adj_graph <= th] = 0
# adj_graph = adj_graph / np.linalg.norm(adj_graph)
# adj_graph = adj_graph[:, np.argsort(adj_graph)[:]]
# adj_graph[1]

In [41]:
class_ids = class_id_emb_attr.class_id.values
class_to_id = dict([(c, i) for i, c in enumerate(class_ids)])
# class_to_id

In [35]:
BATCH_SIZE = 32

# adj_graph = 1 - sklearn.metrics.pairwise.pairwise_distances(
#     np.array(list(class_id_emb_attr['emb']))[:, :300], metric = 'cosine')

def create_gcn():
    alpha = 0.03
    img_flat_len = 1032
    attr_input = Input(tensor=tf.constant(np.array(list(class_id_emb_attr['attr']), dtype = 'float32')))
    all_word_emb = Input(tensor=tf.constant(np.array(list(class_id_emb_attr['emb']))[:, :300], dtype = 'float32')) #Input(shape = (230, 300,), name = 'wv')
    class_index = Input(shape = (1, ), name = 'class_index', dtype = 'int32')
    adj_graphs = Input(tensor=tf.constant(adj_graph, dtype = 'float32')) #Input(shape = (230, 230,), name = 'adj_graph')
    imag_classifier = Input(shape = (img_flat_len,), name = 'img')
    
    attr_dense = Dense(300, use_bias = False, kernel_initializer='he_normal', 
                    kernel_regularizer = l2(1e-4))(attr_input)
    attr_word_emb = Concatenate()([all_word_emb, attr_dense])
#     x = Lambda(lambda xx: all_word_emb)(class_index)
#     x = Dense(516, kernel_initializer='he_normal', kernel_regularizer = l2(1e-4), 
#               activation = 'relu', name = 'conv')(all_word_emb)
#     all_classifier = Lambda(lambda x: K.dot(x[1], x[0]), name = 'rela')([x, adj_graphs])
    all_classifier = full_connect_layer(attr_word_emb, hidden_dim = [int(img_flat_len * 1.5), 
                                                                    int(img_flat_len * 1.25 ),
                                                                    img_flat_len], 
                                activation = 'relu', adj_graphs = adj_graphs)
    x = tf.gather_nd(all_classifier, class_index)

    mse_loss = K.mean(keras.losses.mean_squared_error(imag_classifier, x))
    
    model = Model([class_index, imag_classifier, attr_input, all_word_emb, adj_graphs], outputs = [all_classifier]) #, vgg_output])
    model.add_loss(mse_loss)
    model.compile(optimizer=keras.optimizers.Adam(lr=1e-4), loss=None)
#     model.summary()
    return model

kf = KFold(n_splits=5, shuffle=True, random_state = 1)
num_fold = 0
zs_model_list = []
classes = train_data.class_id.unique()
class_ids = class_id_emb_attr.class_id.values
class_to_id = dict([(c, i) for i, c in enumerate(class_ids)])

for train_index, test_index in kf.split(classes):
#     print (train_index)
    seen_class = classes[train_index]
    unseen_class = classes[test_index]
    
    train_part_df = train_data[train_data.class_id.isin(seen_class)]
    validate_part_df = train_data[train_data.class_id.isin(unseen_class)]
    
    train_part_data = np.array([class_to_id[c] for c in train_part_df['class_id'].values]).astype('int32')
    validate_part_data = np.array([class_to_id[c] for c in validate_part_df['class_id'].values]).astype('int32')

    train_part_target = extract_array_from_series(train_part_df['target'])
    validate_part_target = extract_array_from_series(validate_part_df['target'])

#     print ('Train Validation Classes: ', train_part_df.class_id.unique().shape[0], 
#            validate_part_df.class_id.unique().shape[0])
    print ('Seen unseen Classes: ', seen_class.shape[0], unseen_class.shape[0])

    callbacks = [
            EarlyStopping(monitor='val_loss', patience=50, verbose=0),
            AccuracyEvaluation(validation_data=(validate_part_data,  validate_part_target), interval=1, \
    #                         class_id_emb_attr = class_id_emb_attr, \
                               eval_df = validate_part_df, threshold= 0.3, \
                              seen_class = seen_class, unseen_class = unseen_class, \
                class_id_emb_attr = class_id_emb_attr[class_id_emb_attr.class_id.isin(unseen_class)], \
                              gamma = 0.8, model_type = 'GCN')
            ]
    zs_model = create_gcn()
    if num_fold == 0:
        print (zs_model.summary())
    zs_model.fit([train_part_data, train_part_target],  
                 validation_data = ([validate_part_data, validate_part_target], None),
                  epochs=25, batch_size = 32, shuffle=True, verbose = 1, callbacks=callbacks)
    zs_model_list.append((zs_model, 'GCN'))
    num_fold += 1

Seen unseen Classes:  164 41
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_32 (InputLayer)            (285, 30)             0                                            
____________________________________________________________________________________________________
input_33 (InputLayer)            (285, 300)            0                                            
____________________________________________________________________________________________________
dense_84 (Dense)                 (285, 300)            9000        input_32[0][0]                   
____________________________________________________________________________________________________
concatenate_15 (Concatenate)     (285, 600)            0           input_33[0][0]                   
                                                              

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:32: UserWarning: Output "rela_2" missing from loss dictionary. We assume this was done on purpose, and we will not be expecting any data to be passed to "rela_2" during training.


Train on 69913 samples, validate on 17336 samples
Epoch 1/25
69792/69913 [============================>.] - ETA: 0s - loss: 1.7330eval img id:  ZJL263 has multiple best candidates:  2 min val:  18.49614

All_re: 	0.038417	666	17336
Seen_re: 	nan	0	0
Unseen_re: 	0.038417	666	17336
69913/69913 [==============================] - 33s - loss: 1.7317 - val_loss: 1.0597
Epoch 2/25
  416/69913 [..............................] - ETA: 29s - loss: 0.9983

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:41: RuntimeWarning: invalid value encountered in long_scalars


69856/69913 [============================>.] - ETA: 0s - loss: 0.9736
All_re: 	0.068066	1180	17336
Seen_re: 	nan	0	0
Unseen_re: 	0.068066	1180	17336
69913/69913 [==============================] - 32s - loss: 0.9736 - val_loss: 1.0059
Epoch 3/25
69792/69913 [============================>.] - ETA: 0s - loss: 0.8904
All_re: 	0.082603	1432	17336
Seen_re: 	nan	0	0
Unseen_re: 	0.082603	1432	17336
69913/69913 [==============================] - 32s - loss: 0.8903 - val_loss: 0.9059
Epoch 4/25
 9536/69913 [===>..........................] - ETA: 24s - loss: 0.8300

KeyboardInterrupt: 

In [103]:
multi_models_vote(models = zs_model_list, eval_df = validate_part_df, \
            cand_class_id_emb_attr = class_id_emb_attr[class_id_emb_attr.class_id.isin(unseen_class)], \
            img_feature_map = validate_part_target, seen_class = seen_class, unseen_class = unseen_class)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:41: RuntimeWarning: invalid value encountered in long_scalars



All_re: 	0.156834	2140	13645
Seen_re: 	nan	0	0
Unseen_re: 	0.156834	2140	13645

All_re: 	0.155148	2117	13645
Seen_re: 	nan	0	0
Unseen_re: 	0.155148	2117	13645

All_re: 	0.151118	2062	13645
Seen_re: 	nan	0	0
Unseen_re: 	0.151118	2062	13645

All_re: 	0.153316	2092	13645
Seen_re: 	nan	0	0
Unseen_re: 	0.153316	2092	13645

All_re: 	0.157640	2151	13645
Seen_re: 	nan	0	0
Unseen_re: 	0.157640	2151	13645
[['ZJL264' 'ZJL264' 'ZJL264' 'ZJL264' 'ZJL264']
 ['ZJL102' 'ZJL102' 'ZJL102' 'ZJL102' 'ZJL102']
 ['ZJL254' 'ZJL276' 'ZJL254' 'ZJL254' 'ZJL254']
 ...
 ['ZJL254' 'ZJL254' 'ZJL276' 'ZJL254' 'ZJL254']
 ['ZJL276' 'ZJL276' 'ZJL276' 'ZJL276' 'ZJL254']
 ['ZJL168' 'ZJL125' 'ZJL50' 'ZJL168' 'ZJL254']]
['ZJL264' 'ZJL102' 'ZJL254' ... 'ZJL254' 'ZJL276' 'ZJL168']

All_re: 	0.166215	2268	13645
Seen_re: 	nan	0	0
Unseen_re: 	0.166215	2268	13645


In [48]:
with open(path + 'model_sub/train_data_2018_09_23_10_32_21.pickle', 'rb') as handle:
    test = pickle.load(handle)

In [50]:
test.head()

,class_id,img_id,img,target,preds
0,ZJL1,a6394b0f513290f4651cc46792e5ac86.jpeg,"[[[20.0, 33.0, 7.0], [19.0, 32.0, 6.0], [22.0,...","[2.2266192, 0.16327204, 0.2838421, 0.2219766, ...","[0.9520665, 4.647786e-09, 7.417136e-08, 3.9506..."
1,ZJL1,2fb89ef2ace869d3eb3bdd3afe184e1c.jpeg,"[[[140.0, 45.0, 51.0], [144.0, 47.0, 54.0], [1...","[0.17702743, 0.55263615, 0.0, 0.030876435, 1.2...","[1.30949e-05, 5.735788e-10, 6.5571693e-12, 5.9..."
2,ZJL1,eda9f3bef2bd8da038f6acbc8355fc25.jpeg,"[[[81.0, 69.0, 21.0], [86.0, 74.0, 26.0], [85....","[0.0, 0.42584094, 0.0, 0.034428038, 0.527664, ...","[0.9993531, 4.2289447e-09, 1.2198088e-07, 3.68..."
3,ZJL1,7d93ef45972154aae150b4f9980a79c0.jpeg,"[[[16.0, 14.0, 15.0], [18.0, 16.0, 17.0], [19....","[0.6332717, 0.23473893, 0.0, 0.779357, 1.38354...","[0.9999671, 4.578459e-10, 6.6002594e-12, 8.168..."
4,ZJL1,fb901b4f9a8e396c1d0155bccc5e5671.jpeg,"[[[120.0, 124.0, 127.0], [89.0, 93.0, 96.0], [...","[0.37087774, 1.1033719, 0.0, 0.23497638, 3.003...","[0.86532485, 3.0740804e-08, 3.727919e-08, 1.21..."


In [23]:
test_data = pd.read_csv(path + '/DatasetA_test_20180813/DatasetA_test/image.txt', header = None, names = ['img_id'])
imag_path = path + '/DatasetA_test_20180813/DatasetA_test/test/'
test_data['img'] = test_data['img_id'].apply(lambda id: read_image(imag_path, id))
# with open(path + 'test_data.pickle', 'rb') as handle:
#     test_data = pickle.load(handle)

In [79]:
# with open(path + 'test_data.pickle', 'wb+') as handle:
#     pickle.dump(test_data, handle)
# test_data.head()
with open(path + '/model_sub/stacking_train_label_2018_09_12_12_10_06.pickle', 'rb') as handle:
    stacking_train_data = pickle.load(handle)
stacking_train_data.head()

0        [0.12423625, 0.1190791, 0.7566846]
1       [1.7260123e-13, 2.1865514e-08, 1.0]
2       [0.2117803, 0.17989996, 0.60831976]
3        [0.2420589, 0.17926142, 0.5786797]
4    [0.041276723, 0.95284086, 0.005882429]
dtype: object

In [24]:
img_model = DenseNet(blocks = [6, 12, 24, 16], 
                                cat_max = 171,
                                weight_decay = 1e-4, 
                                kernel_initializer = 'glorot_normal',
                                reduction = 0.5, 
                                init_filters = 64, 
                                growth_rate = 16).model
img_model.load_weights(path + 'model_sub/6_12_24_16_ini64_growth16_02962/model_0_2018_09_13_08_23_48.h5')
img_model_flat = Model(input = img_model.input, output = img_model.get_layer(name = 'avg_pool').output)

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_17 (InputLayer)            (None, 64, 64, 3)     0                                            
____________________________________________________________________________________________________
zero_padding2d_3 (ZeroPadding2D) (None, 70, 70, 3)     0           input_17[0][0]                   
____________________________________________________________________________________________________
conv1/conv (Conv2D)              (None, 68, 68, 64)    1728        zero_padding2d_3[0][0]           
____________________________________________________________________________________________________
conv1/bn (BatchNormalization)    (None, 68, 68, 64)    256         conv1/conv[0][0]                 
___________________________________________________________________________________________

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:9: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("av...)`
  if __name__ == '__main__':


In [96]:
# test_data = setB_test_data
# # test_img = extract_array_from_series(test_data['img'])
# # test_img = vgg16.preprocess_input(test_img)
# # test_img_feature_map = img_model_flat.predict(test_img, verbose = 1)

# with open(path + '/model_sub/6_12_24_16_ini64_growth32_inistride1_augdata_05010/flat_test_re_2018_09_22_19_24_38.pickle', 'rb') as handle:
#     test_img_feature_map = pickle.load(handle)
train_id = train_data['class_id'].unique()
test_img_feature_map = extract_array_from_series(test_data['target'])
# class_id_emb_attr.reset_index(inplace = True)

In [107]:
zs_partial_model = Model(inputs = zs_model.inputs[2:], outputs = zs_model.outputs[0])
test_class_id_emb_attr = class_id_emb_attr #[~class_id_emb_attr['class_id'].isin(train_id)]
# pred_nearest_class_id = find_nearest_class(test_class_id_emb_attr, zs_partial_model, test_data, test_img_feature_map)

In [97]:
pred_nearest_class_id = multi_models_vote(models = zs_model_list, eval_df = test_data, \
            cand_class_id_emb_attr = class_id_emb_attr[~class_id_emb_attr['class_id'].isin(train_id)], \
            img_feature_map = test_img_feature_map)

[['ZJL272' 'ZJL224' 'ZJL242' 'ZJL243' 'ZJL272']
 ['ZJL243' 'ZJL288' 'ZJL239' 'ZJL243' 'ZJL239']
 ['ZJL255' 'ZJL255' 'ZJL255' 'ZJL255' 'ZJL253']
 ...
 ['ZJL286' 'ZJL259' 'ZJL249' 'ZJL259' 'ZJL287']
 ['ZJL270' 'ZJL253' 'ZJL253' 'ZJL253' 'ZJL253']
 ['ZJL270' 'ZJL288' 'ZJL288' 'ZJL288' 'ZJL253']]
['ZJL272' 'ZJL239' 'ZJL255' ... 'ZJL259' 'ZJL253' 'ZJL288']


In [98]:
time_label = time.strftime('_%Y_%m_%d_%H_%M_%S', time.gmtime())
sub_name = path + '/model_sub/sub' + time_label + '.txt'
sub = pd.DataFrame(pred_nearest_class_id, index = test_data['img_id'])
sub.to_csv(sub_name, header = False, sep = '\t')
# zs_model.save(path + 'zs_model' + time_label + '.h5')

In [28]:
152 * 200 / 3600

8.444444444444445